## Definite SMPLX class for SMPLX model

cette parrtie sert a reccuperer les donnés de laa base de donnné CMU pour ensiute les traiter

In [ ]:
import torch
from body_model import BodyModel
from utils import rodrigues_2_rot_mat  
from lbs import lbs, batch_rodrigues  
import os
import numpy as np

# -------------------------------------------------------------
# recuperation du fichier MALE SMPLX 
# /!\ Attention : cela depend de l'arborescence de votre projet
# -------------------------------------------------------------

# On remonte d’un niveau dans l’arborescence des dossiers à partir du répertoire courant
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Chemin absolu vers le fichier du modèle SMPLX
SMPLX_MODEL_MALE_PATH = os.path.join(
    PROJECT_ROOT,
    'test smplx/smplx_lockedhead_20230207/models_lockedhead/smplx/SMPLX_MALE.npz'
)



class SMPLX(BodyModel):
    def __init__(self, num_betas=16, **kwargs):
        # Initialisation du modèle SMPLX à partir du fichier .npz
        super().__init__(
            bm_fname=SMPLX_MODEL_MALE_PATH,
            num_betas=num_betas,
            num_expressions=0,  
            **kwargs
        )

    def forward(self, pose_body, betas, use_rodrigues=True):
        """
        Génère une sortie 3D (vertices, joints, faces) à partir d'un vecteur de pose et de paramètres de forme.
        
        pose_body : [B, 66]  -> 22 articulations * 3 (Rodrigues ou rot6d)
        betas     : [B, 16]  -> Coefficients de forme
        """

        device = pose_body.device

        
        for name in [
            'init_pose_hand', 'init_pose_jaw', 'init_pose_eye',
            'init_v_template', 'init_expression', 'shapedirs',
            'exprdirs', 'posedirs', 'J_regressor',
            'kintree_table', 'weights', 'f'
        ]:
            tensor = getattr(self, name)
            setattr(self, name, tensor.to(device))

        batch_size = pose_body.shape[0]

        # Initialisation des poses neutres pour les mains, la mâchoire, les yeux
        pose_hand = self.init_pose_hand.expand(batch_size, -1)
        pose_jaw = self.init_pose_jaw.expand(batch_size, -1)
        pose_eye = self.init_pose_eye.expand(batch_size, -1)

        # Template de vertices et d'expression neutre
        v_template = self.init_v_template.expand(batch_size, -1, -1)
        expression = self.init_expression.expand(batch_size, -1)

        
        init_pose = torch.cat([pose_jaw, pose_eye, pose_hand], dim=-1)  # [B, 99]
        if not use_rodrigues:
            init_pose = rodrigues_2_rot_mat(init_pose)  # [B, 33, 3, 3]

        full_pose = torch.cat([pose_body, init_pose], dim=-1)  # [B, 165]

        shape_components = torch.cat([betas, expression], dim=-1)

        shapedirs = torch.cat([self.shapedirs, self.exprdirs], dim=-1)

        # ------------------------------
        # Appel au Linear Blend Skinning (déformation du maillage)
        # ------------------------------
        verts, joints = lbs(
            betas=shape_components,
            pose=full_pose,
            v_template=v_template,
            shapedirs=shapedirs,
            posedirs=self.posedirs,
            J_regressor=self.J_regressor,
            parents=self.kintree_table[0].long(),
            lbs_weights=self.weights,
            pose2rot=use_rodrigues
        )

        # Récupération des faces du mesh, répétées pour chaque instance du batch
        faces = self.f.expand(batch_size, -1, -1)


        return dict(verts=verts, faces=faces, joints=joints)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
smplx = SMPLX().to(device)

### Fonctions pour visualiser le body : une qui permet de visualiser les frame et une qui visualise l'animation 

In [ ]:
# visualiser le nuage de points avant et après translation

from matplotlib import pyplot as plt

def visualize_frame(verts, trans):
    # visualiser le mesh
    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    ax1.scatter(verts[:, 0], verts[:, 1], verts[:, 2], c='gray', marker='o', s=0.01)
    ax1.set_title("Mesh before translation")
    val_min = verts.min()
    val_max = verts.max()
    ax1.set_xlim(val_min, val_max)
    ax1.set_ylim(val_min, val_max)
    ax1.set_zlim(val_min, val_max)


    verts_ = verts + trans 
    ax2.scatter(verts_[:, 0], verts_[:, 1], verts_[:, 2], c='gray', marker='o', s=0.01)
    ax2.set_title("Mesh after translation")
    val_min = verts_.min()
    val_max = verts_.max()
    ax2.set_xlim(val_min, val_max)
    ax2.set_ylim(val_min, val_max)
    ax2.set_zlim(val_min, val_max)

In [ ]:
# visualiser la suite de nuages de points et de meshes de mouvement

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def visualize_seq(verts, faces):
    fig = plt.figure(figsize=(12, 6))
    ax_pc = fig.add_subplot(121, projection='3d')
    ax_mesh = fig.add_subplot(122, projection='3d')

    total_frames = len(verts)

    def update_view(frame):
        ax_pc.cla()
        ax_mesh.cla()

        val_min = verts.min()
        val_max = verts.max()

        # Extrait le nuage de points et theta pour la frame actuelle
        gt_pc = verts[frame].cpu().numpy()

        # affiche le nuage de points
        ax_pc.scatter(gt_pc[:, 0], gt_pc[:, 1], gt_pc[:, 2], s=0.01, c='gray')
        ax_pc.set_title(f"Point Cloud Frame {frame + 1}")
        ax_pc.set_xlim(val_min, val_max)
        ax_pc.set_ylim(val_min, val_max)
        ax_pc.set_zlim(val_min, val_max)
        ax_pc.view_init(elev=20, azim=50)


        # affiche le mesh
        ax_mesh.plot_trisurf(gt_pc[:, 0], gt_pc[:, 1], gt_pc[:, 2], triangles=faces, color='green', alpha=0.2)
        ax_mesh.set_title(f"Mesh Frame {frame + 1}")
        ax_mesh.set_xlim(val_min, val_max)
        ax_mesh.set_ylim(val_min, val_max)
        ax_mesh.set_zlim(val_min, val_max)
        ax_mesh.view_init(elev=20, azim=50)

    #créé une animation pour toutes les frames de tous les batches
    ani = FuncAnimation(fig, update_view, frames=total_frames)
    plt.close(fig)  # ferme la figure pour eviter d'afficher un plot immobile 

    return ani


### Charge les data CMU et créé le Mesh en uttilisant le model SMPLX

In [ ]:
# CHARGER LE JEU DE DONNÉES CMU
import numpy as np
import matplotlib.pyplot as plt

# Charger le fichier npz
npz_file_path = "CMU/01/01_02_poses.npz"
npz_smplx = "smplx_lockedhead_20230207/models_lockedhead/smplx/SMPLX_MALE.npz"
npz_file = np.load(npz_file_path)#, allow_pickle=True)
npz_file_smplx = np.load(npz_smplx, allow_pickle=True)  

# Extraire les données
print("Extraction des données...")
data = {}
for key in npz_file.keys():
    print(f"Clé : {key}")
    try : 
        if npz_file[key].shape == ():
            print(f"Contenu : {npz_file[key]}")
        else:
            print(f"Forme : {npz_file[key].shape}")
    except:
        continue
    data[key] = npz_file[key]
    print(data[key].shape)
print("Données extraites.")


In [ ]:
# Créer les MAILLAGES
pose_body = torch.tensor(data['poses'][:-1, :66], dtype=torch.float32).to(device)  # 21 articulations (63 vecteurs)
beta = torch.tensor(data['betas'], dtype=torch.float32).unsqueeze(0).repeat(pose_body.shape[0], 1).to(device)

meshes = smplx(pose_body=pose_body, betas=beta, use_rodrigues=True)
verts = meshes['verts']  # .verts_padded()
faces = meshes['faces']  # .faces_packed()
trans = data['trans']


### VISUALISER UNE POSE 

In [ ]:
# VUSUALISER UNE POSE 
f = 0
visualize_frame(verts[f], trans[f])

In [ ]:
# VISUALISER UNE SUITE

train_animation = visualize_seq(verts[:3000:40], faces[0])
display(HTML(train_animation.to_jshtml()))

In [ ]:
def visualize_seq_with_translation(verts, faces, trans):
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(12, 6))
    ax_pc = fig.add_subplot(121, projection='3d')
    ax_mesh = fig.add_subplot(122, projection='3d')

    total_frames = len(verts)

    def update_view(frame):
        ax_pc.cla()
        ax_mesh.cla()

        val_min=verts.min().item()-0.7
        val_max=verts.max().item()+0.7


        # Apply translation to the current frame
        translated_verts = verts[frame] + trans[frame]

        gt_pc = translated_verts.cpu().numpy()

        # Point cloud
        ax_pc.scatter(gt_pc[:, 0], gt_pc[:, 1], gt_pc[:, 2], s=0.01, c='gray')
        ax_pc.set_title(f"Point Cloud Frame {frame + 1}")
        ax_pc.set_xlim(val_min, val_max)
        ax_pc.set_ylim(val_min, val_max)
        ax_pc.set_zlim(val_min, val_max)
        ax_pc.view_init(elev=20, azim=50)

        # Mesh
        ax_mesh.plot_trisurf(gt_pc[:, 0], gt_pc[:, 1], gt_pc[:, 2], triangles=faces, color='green', alpha=0.2)
        ax_mesh.set_title(f"Mesh Frame {frame + 1}")
        ax_mesh.set_xlim(val_min, val_max)
        ax_mesh.set_ylim(val_min, val_max)
        ax_mesh.set_zlim(val_min, val_max)
        ax_mesh.view_init(elev=20, azim=50)

    ani = FuncAnimation(fig, update_view, frames=total_frames)
    plt.close(fig)
    return ani

# 2. CODE POUR LIRE LES DONNÉS CMU

### Load one sequence of CMU data 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the npz file
npz_file_path = "CMU/01/01_02_poses.npz"
npz_smplx = "smplx_lockedhead_20230207/models_lockedhead/smplx/SMPLX_MALE.npz"
npz_file = np.load(npz_file_path)#, allow_pickle=True)
npz_file_smplx = np.load(npz_smplx, allow_pickle=True)  

# Extract the data
print("Extracting data...")
data = {}
for key in npz_file.keys():
    print(f"Key: {key}")
    try : 
        if npz_file[key].shape == ():
            print(f"Content: {npz_file[key]}")
        else:
            print(f"Shape: {npz_file[key].shape}")
    except:
         
        continue
    data[key] = npz_file[key]
    print(data[key].shape)
print("Data extracted.")

### RECUPERER LA TRANSLATATION 

In [ ]:
trans = data['trans']

### Visualier la transation 

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(trans[:,0], trans[:,1], trans[:,2], s=1, label='translation')

In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt

from openai import OpenAI


text = "The boy goes to the sofa, then he sits on the chair and finally he lays on the bed."


client = OpenAI(
 base_url="https://openrouter.ai/api/v1",
 api_key="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",
)


completion = client.chat.completions.create(
 model="cognitivecomputations/dolphin3.0-r1-mistral-24b:free",
 messages=[
   {
     "role": "user",
     "content": "You are an assistant that helps people find objects in a room. You should determine targets objects in the text description that we will give you."},
               {
                   "role": "assistant",
                   "content": """
                       Here are the examples:
                     
                       Please note that targets should be split by ",".
                       1. Walk next to the bed and then sit on the chair. Please answer:
                           bed,chair
                       2. Sit on the sofa that is next to the table and head to the door. Please answer:
                           sofa,table,door
                       3. sit on the chair that is next to the table. Please answer:
                           chair
                       4. Stand up from the chair that is next to the tables and go to the toilet. Please answer:
                           toilet
                   """,
               },
               {
                   "role": "user",
                   "content": f"""
                       The text description is: {text}.
                       Answer in English.
                       Answer should be in the following format without any explanations: target_object_1, target_object_2, target_object_3, ....
                       Don't give the anchor object, only targets, if an object is used to find the position of an another one, don't give it like exemple 3.
                       If there is no object in the text, return : error 
                   """,
               }
           ]
)


answer=completion.choices[0].message.content
print(answer)
names = answer.split(",")
print(names)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Chargement des fichiers de données .npz :
# - npz_file contient les poses du dataset CMU
# - npz_file_smplx contient les paramètres du modèle SMPLX
# ------------------------------------------------------------

npz_file_path = "CMU/01/01_02_poses.npz"
npz_smplx = "smplx_lockedhead_20230207/models_lockedhead/smplx/SMPLX_MALE.npz"
npz_file = np.load(npz_file_path)  # Chargement du fichier de poses
npz_file_smplx = np.load(npz_smplx, allow_pickle=True)  # Chargement du modèle SMPLX

# ------------------------------------------------------------
# Extraction des données du fichier de poses CMU
# ------------------------------------------------------------

print("Extraction des données...")
data = {}
for key in npz_file.keys():
    print(f"Clé : {key}")
    try:
        if npz_file[key].shape == ():  
            print(f"Contenu : {npz_file[key]}")
        else:  # Si tableau
            print(f"Forme : {npz_file[key].shape}")
    except:
        continue
    data[key] = npz_file[key]
    print(data[key].shape)
print("Données extraites.")

# ------------------------------------------------------------
# Analyse des translations :
# - On récupère la liste des translations 'trans'
# - On calcule les coordonnées cumulées (x, y, z)
# - On détermine des positions de référence espacées régulièrement
# - On stocke aussi les déplacements relatifs entre chaque frame
# ------------------------------------------------------------

coor = []           # Liste des positions cumulées toutes les n frames
lcoor = []          # Liste temporaire pour stocker les difference de positions
lfinale = []        # Liste de toutes les difference de positions, regroupées par segment
trans = data['trans']  # Données de translation donné par smpl-X
ltransitions = []   # Liste des transitions relatives frame à frame

x = 0
y = 0
z = 0
nbpoints = 2  # Nombre de points de référence correspondant au objet de la scene.
ecart = len(trans) // nbpoints  # Espacement entre chaque point

for i in range(1, len(trans) - 1):
    x += trans[i][0]
    y += trans[i][1]
    z += trans[i][2]
    
    # Calcul du déplacement relatif entre la frame suivante et la position cumulée actuelle
    ltransitions.append((
        trans[i+1][0] - x,
        trans[i+1][1] - y,
        trans[i+1][2] - z
    ))

    # Sauvegarde de la translation actuelle
    lcoor.append(np.array(trans[i]).tolist())

    # Toutes les 'ecart' frames, on sauvegarde la position cumulée et les coordonnées du segment
    if i % ecart == 0 and i != 0:
        coor.append((x, y, z))
        lfinale.append(lcoor)
        lcoor = []

# Les listes suivantes contiennent :
# - coor : positions cumulées à intervalles réguliers
# - lfinale : sous-listes des differences de positions par segment
# - ltransitions : vecteurs de transition frame à frame


In [ ]:
import json


def norme(l):
  return ( l[0]**2 + l[1]**2 + l[2]**2)


temps = []
init = 0
fin = 0
for i in range(len(coor)):
  fin += len(lfinale[i])
  temps.append((init, fin-1))
  init = fin


# Créer un dictionnaire Python
donnees = {
  "nom": [],
  "coordonnees": [],
  "temps": [],
  "vecteurs": [],
}


donnees["nom"] = names
donnees["coordonnees"] = coor
donnees["temps"] = temps
donnees["vecteurs"] = lfinale


# Sauvegarder dans un fichier JSON
with open("fichier.json", "w", encoding="utf-8") as f:
  json.dump(donnees, f, indent=4)







# travail final 

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


# ------------------------------------------------------------
# Définition d'une classe Dataset personnalisée pour PyTorch
# Elle permet d'entraîner un modèle à partir d'une séquence de coordonnées et de labels associés
# ------------------------------------------------------------
class CoordinateSequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]
    

# ------------------------------------------------------------
# Fonction de préparation des données d'entrée
# Convertit simplement une liste en tenseur PyTorch
# ------------------------------------------------------------
def setup_data(input_list):
    set_sequences = torch.tensor(input_list, dtype=torch.float32)
    return set_sequences


# ------------------------------------------------------------
# Préparation des données d'entrée (vecteurs) et des labels (poses SMPLX)
# ------------------------------------------------------------

print(len(donnees["vecteurs"]))
print(data["trans"].shape)
print(donnees["vecteurs"])

n_segment = len(donnees["vecteurs"][0])  # Nombre de segments (longueur des séquences)
segment_length = len(data["poses"]) // n_segment  # Nombre de poses par segment
print(segment_length)

# Tronque les poses pour que leur longueur soit un multiple du nombre de segments
truncated_poses = data["poses"][:segment_length * n_segment, :66]

# Reformate les poses en segments de taille fixe (ici on ne garde que le premier segment avec [:1])
reshaped_poses = truncated_poses.reshape(segment_length, n_segment, 66)[:1]

# Conversion des listes en tenseurs PyTorch
input_data = setup_data(donnees["vecteurs"])
labels = setup_data(reshaped_poses)

print(input_data.shape)
print(labels.shape)


# ------------------------------------------------------------
# Préparation des ensembles d'entraînement (ici, pas de test split activé)
# ------------------------------------------------------------

# train_seq, test_seq, train_labels, test_labels = train_test_split(
#     input_data, labels, test_size=0.2, random_state=42
# )
train_seq, train_labels = input_data, labels

# Création des datasets
train_dataset = CoordinateSequenceDataset(train_seq, train_labels)
# test_dataset = CoordinateSequenceDataset(test_seq, test_labels)

# Chargement des données par lot (batch)
train_loader = DataLoader(train_dataset, batch_size=5000, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Choix du périphérique d'entraînement (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ------------------------------------------------------------
# Paramètres du modèle
# ------------------------------------------------------------
sequence_length = len(donnees["vecteurs"][0])  # Longueur des séquences d'entrée
input_size = 3                                # Chaque vecteur a 3 dimensions (x, y, z)
hidden_size = 100                             # Taille de l'état caché de la GRU
output_size = 66                              # Nombre de valeurs de sortie (correspond à la pose)
num_epochs = 50
batch_size = 5000
learning_rate = 0.01
num_layers = 2                                # Nombre de couches dans la GRU


# ------------------------------------------------------------
# Définition du modèle GRU
# Une couche GRU suivie d'une couche linéaire
# ------------------------------------------------------------
class myGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(myGRU, self).__init__()
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialisation de l'état caché à zéro (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(device)

        # Propagation avant dans la GRU
        out, _ = self.rnn(x, h0)  # out : (batch, seq_len, hidden_size)

        # Application de la couche linéaire à chaque pas de temps
        out = self.fc(out)  # out : (batch, seq_len, output_size)
        return out

# Instanciation du modèle et déplacement sur le bon périphérique
model = myGRU(input_size, hidden_size, num_layers, output_size).to(device)

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# ------------------------------------------------------------
# Boucle d'entraînement du modèle
# ------------------------------------------------------------
n_total_steps = len(train_loader)
print(len(train_loader))
for epoch in range(num_epochs):
    for i, (sequences, targets) in enumerate(train_loader):
        sequences = sequences.to(device)
        targets = targets.to(device)
        
        # Prédiction
        outputs = model(sequences)

        # Calcul de la perte
        loss = criterion(outputs, targets)
        
        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Affichage de la progression
        if (i + 1) % 1 == 0:
            print(f'Époque [{epoch+1}/{num_epochs}], Étape [{i+1}/{n_total_steps}], Perte : {loss.item():.4f}')


In [ ]:
# Create MESHES 

poses = model(setup_data(donnees["vecteurs"]))[0]
pose_body = torch.tensor(poses[:-1,:66], dtype=torch.float32).to(device) # 21 joints (63 vec) 
beta = torch.tensor(data['betas'], dtype=torch.float32).unsqueeze(0).repeat(pose_body.shape[0], 1).to(device) 

meshes = smplx(pose_body=pose_body, betas=beta, use_rodrigues=True)
verts = meshes['verts']#.verts_padded()
faces = meshes['faces']#.faces_packed()
trans = data['trans']

In [ ]:
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 100  # autorise les grandes animations

trans_tensor = torch.tensor(data['trans'], dtype=torch.float32).to(device)
ani = visualize_seq_with_translation(verts[::40], faces[0].cpu().numpy(), trans_tensor[::40])
HTML(ani.to_jshtml())